<a href="https://colab.research.google.com/github/pankaj-inbetween/search-in-video/blob/main/Search_In_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pysrt langchain_astradb langchain_openai
import pysrt
import datetime
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.1 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'dotenv'

In [ ]:
load_dotenv()

In [ ]:
subs = pysrt.open('sub.srt')
youtube_link = "https://www.youtube.com/watch?v=haDg_l9jjxA"

In [ ]:
def split_by_timewindow(subs, window_in_minutes):
    docs = []
    t = ''
    split_time = datetime.time(hour=0,minute=window_in_minutes, second=0)
    for s in subs:
        if s.start.to_time() < split_time:
            t = t + ' ' + s.text
        else:
            start_time =  (split_time.hour*3600) + (split_time.minute*60) + (split_time.second) - 60
            docs.append(Document(page_content=t,metadata={"link":f"{youtube_link}&t={start_time}s", "start_time":start_time}))
            t = s.text
            split_time = (datetime.datetime(1, 1, 1,split_time.hour,split_time.minute, split_time.second) +  datetime.timedelta(minutes=window_in_minutes)).time()
    return docs

In [ ]:
t=split_by_timewindow(subs, 1)
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="test",
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
)

In [ ]:
vstore.add_documents(t)

In [ ]:
result = vstore.similarity_search_with_relevance_scores("bio technology application in agriculture", k=5)

data = { "subtitle": [], "link": [], "start_time": [], "score": []}
for r in result:
    data['subtitle'].append(r[0].page_content)
    data['link'].append(r[0].metadata['link'])
    data['score'].append(r[1])
    data['start_time'].append(r[0].metadata['start_time'])
df = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)
df


,subtitle,link,start_time,score
0,whatever organic compounds are there we will isolate them now biotechnology application in agriculture there are various uh you know applications are there see with increasing population like we have read in the chapter number two of zoology which is reproductive Health there was a major issue and that major issue was related to the population increasing you know population in millions it was difficult to feed those number of individuals so what we did we tried to increase the agriculture right by that only we can provide them food so we started using some chemicals we started using some chemicals like insecticide and pesticides and we when we started using those insecticide and pesticide what happened production increased and because of that insecticide and pesticide they had some residual effects also though this lead to the Green Revolution also which later we realized,https://www.youtube.com/watch?v=haDg_l9jjxA&t=21180s,21180,0.943823
1,we are starting with the biotechnology,https://www.youtube.com/watch?v=haDg_l9jjxA&t=16920s,16920,0.937256
2,pesticide we don't want pesticides help to reduce post Harvest loss when we are storing those food in bulk nothing can happen to them next increased efficiency of mineral usage by plants more and more plants are extracting nutrients from the soil even when they are at dwarf prod you know when when they are small and high they're producing fruit how it is possible bi technology only enhanced nutritional value for example today we have produced golden rice golden rice means doesn't means it is golden it's made up of gold it doesn't mean that golden rice means it is yellow color rice which is rich in vitamin A okay so with the help of all these things today all biotechnology we have made these things possible so everybody my cbsc aspirant keep these points in your mind okay now what we have prepared so far let's talk,https://www.youtube.com/watch?v=haDg_l9jjxA&t=21300s,21300,0.928397
3,their natural capability now what we are doing is with the help of a recombinant DNA technology we are modifying those microorganism so that we can use them for the products and the services let's have a look so what is a standard definition of biotechnology given by efb that is the European Federation of biotechnology European Federation of biotechnology says when we integrate when we use the natural capability that means natural sign means organism sells part of their molecular analog for our products and services this is termed as the biotechnology let me first switch on the AC one second it's really very hard and guys I'm telling you seriously it's so difficult to stand in front of the screen with just warit heat one,https://www.youtube.com/watch?v=haDg_l9jjxA&t=17340s,17340,0.921951
4,chapter it's not that big one it's a small as compared to the uh you know the other chapters so let's do this biot technology application so you know the biotechnology biotechnology is we are modifying it we are using genetic engineering and we are using bioprocesses yes then previously other chapters whatever is there now application of biotechnology there are various applications of biotechnology one is Therapeutics diagnostic genetically modified crop for agriculture processed food bior remediation waste material removal and the energy production these are the different ways right okay three critical areas of the research in biotechnology are see biotechnology what we are doing is we are modifying something right we are modifying something here it provide the,https://www.youtube.com/watch?v=haDg_l9jjxA&t=21060s,21060,0.921778
